In [ ]:
from PIL import Image
import pandas as pd
import numpy as np
import os

# Initialize empty lists to store images and labels
loaded_images = []
loaded_labels = []

# Load the CSV file into a DataFrame
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/tag_clean_10_6.csv')

# Create a dictionary for filename-label mapping
filename_label_mapping = dict(zip(df['filename'], df['tag']))

# Specify the folder where the preprocessed images are saved
save_folder = '/content/drive/MyDrive/Colab Notebooks/preprocessed_images/'

# Loop through the saved images and load them into memory
for filename in os.listdir(save_folder):
    if filename.endswith(".png"):  # Add more conditions if you have multiple file types
        img_path = os.path.join(save_folder, filename)

        # Open and read the image
        image_data = Image.open(img_path)

        # Convert the image to a numpy array and normalize it
        image_data = np.array(image_data) / 255.0

        # Append the image data to the list
        loaded_images.append(image_data)

        # Use filename to get the corresponding label from the mapping
        label = filename_label_mapping.get(filename, None)
        if label is not None:
            loaded_labels.append(label)

# Convert lists to numpy arrays
images = np.array(loaded_images)
labels = np.array(loaded_labels)


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_temp, y_train, y_temp = train_test_split(images, labels, test_size=0.4, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

datagen.fit(X_train)

In [ ]:
print("Distribution of classes in y_train:", np.bincount(y_train))

Distribution of classes in y_train: [  0 155 167 599]


In [ ]:
y_train = y_train - 1
y_val = y_val - 1
y_test = y_test - 1

print("Distribution of classes in y_train:", np.bincount(y_train))

Distribution of classes in y_train: [155 167 599]


In [ ]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D, Dropout, Dense, BatchNormalization, Activation
from tensorflow.keras.optimizers import Adam

# Initialize the base model with pre-trained ResNet50
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(256, 256, 3))

# Freeze the layers of the base model
for layer in base_model.layers:
    layer.trainable = False

# Add custom layers on top of the base model
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dropout(0.5),
    Dense(1024),
    BatchNormalization(),
    Activation('relu'),
    Dropout(0.5),
    Dense(512),
    BatchNormalization(),
    Activation('relu'),
    Dropout(0.5),
    Dense(256),
    BatchNormalization(),
    Activation('relu'),
    Dense(3, activation='softmax')
])

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.00001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])



94765736/94765736 [==============================] - 5s 0us/step


In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger

# Path to save the best model based on validation accuracy
best_model_save_path = '/content/drive/MyDrive/Colab Notebooks/best_model_ResNet50.h5'

# Path to save the CSV log of training history
csv_log_save_path = '/content/drive/MyDrive/Colab Notebooks/training_history_ResNet50.csv'

# Initialize the ModelCheckpoint and CSVLogger callbacks
model_checkpoint = ModelCheckpoint(best_model_save_path,
                                   monitor='val_accuracy',
                                   mode='max',
                                   save_best_only=True,
                                   verbose=1)

csv_logger = CSVLogger(csv_log_save_path, separator=',', append=False)


history = model.fit(datagen.flow(X_train, y_train, batch_size=32),
                    epochs=500,
                    validation_data=(X_val, y_val),
                    callbacks=[model_checkpoint, csv_logger])


model.save('/content/drive/MyDrive/Colab Notebooks/last_model_ResNet50.h5')



Epoch 1/500
29/29 [==============================] - ETA: 0s - loss: 1.6892 - accuracy: 0.2389
Epoch 1: val_accuracy improved from -inf to 0.62866, saving model to /content/drive/MyDrive/Colab Notebooks/best_model_ResNet50.h5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


29/29 [==============================] - 29s 485ms/step - loss: 1.6892 - accuracy: 0.2389 - val_loss: 1.0616 - val_accuracy: 0.6287
Epoch 2/500
29/29 [==============================] - ETA: 0s - loss: 1.7089 - accuracy: 0.2085
Epoch 2: val_accuracy did not improve from 0.62866
29/29 [==============================] - 12s 427ms/step - loss: 1.7089 - accuracy: 0.2085 - val_loss: 1.1254 - val_accuracy: 0.1564
Epoch 3/500
29/29 [==============================] - ETA: 0s - loss: 1.6598 - accuracy: 0.2269
Epoch 3: val_accuracy did not improve from 0.62866
29/29 [==============================] - 12s 424ms/step - loss: 1.6598 - accuracy: 0.2269 - val_loss: 1.2845 - val_accuracy: 0.1401
Epoch 4/500
29/29 [==============================] - ETA: 0s - loss: 1.5829 - accuracy: 0.2356
Epoch 4: val_accuracy did not improve from 0.62866
29/29 [==============================] - 12s 422ms/step - loss: 1.5829 - accuracy: 0.2356 - val_loss: 1.4327 - val_accuracy: 0.1401
Epoch 5/500
29/29 [===============

In [ ]:
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger
from tensorflow.keras.optimizers import Adam

# Path to save the best model based on validation accuracy
best_model_save_path = '/content/drive/MyDrive/Colab Notebooks/best_model_ResNet50.h5'

# Path to save the CSV log of training history
csv_log_save_path = '/content/drive/MyDrive/Colab Notebooks/training_history_ResNet50_3.csv'

# Initialize the ModelCheckpoint and CSVLogger callbacks
model_checkpoint = ModelCheckpoint(best_model_save_path,
                                   monitor='val_accuracy',
                                   mode='max',
                                   save_best_only=True,
                                   verbose=1)

csv_logger = CSVLogger(csv_log_save_path, separator=',', append=False)
# Load the best saved model
best_model_path = '/content/drive/MyDrive/Colab Notebooks/best_model_ResNet50.h5'


# Load the best saved model
model = load_model(best_model_path)

# Compile the model with a new learning rate
new_learning_rate = 0.001  # Replace with your desired rate
model.compile(optimizer=Adam(learning_rate=new_learning_rate),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])


# Continue training
history = model.fit(datagen.flow(X_train, y_train, batch_size=32),
                    epochs=400,  # specify the number of additional epochs
                    validation_data=(X_val, y_val),
                    callbacks=[model_checkpoint, csv_logger])

model.save('/content/drive/MyDrive/Colab Notebooks/last_model_ResNet50.h5')

Epoch 1/400
29/29 [==============================] - ETA: 0s - loss: 1.0256 - accuracy: 0.5809
Epoch 1: val_accuracy improved from -inf to 0.70033, saving model to /content/drive/MyDrive/Colab Notebooks/best_model_ResNet50.h5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


29/29 [==============================] - 28s 494ms/step - loss: 1.0256 - accuracy: 0.5809 - val_loss: 0.8087 - val_accuracy: 0.7003
Epoch 2/400
29/29 [==============================] - ETA: 0s - loss: 1.0054 - accuracy: 0.5972
Epoch 2: val_accuracy did not improve from 0.70033
29/29 [==============================] - 12s 422ms/step - loss: 1.0054 - accuracy: 0.5972 - val_loss: 0.8262 - val_accuracy: 0.7003
Epoch 3/400
29/29 [==============================] - ETA: 0s - loss: 0.9882 - accuracy: 0.6156
Epoch 3: val_accuracy did not improve from 0.70033
29/29 [==============================] - 12s 419ms/step - loss: 0.9882 - accuracy: 0.6156 - val_loss: 0.8390 - val_accuracy: 0.6971
Epoch 4/400
29/29 [==============================] - ETA: 0s - loss: 0.9882 - accuracy: 0.5983
Epoch 4: val_accuracy did not improve from 0.70033
29/29 [==============================] - 12s 415ms/step - loss: 0.9882 - accuracy: 0.5983 - val_loss: 0.9878 - val_accuracy: 0.4593
Epoch 5/400
29/29 [===============

In [ ]:
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger
from tensorflow.keras.optimizers import Adam

# Path to save the best model based on validation accuracy
best_model_save_path = '/content/drive/MyDrive/Colab Notebooks/best_model_ResNet50.h5'

# Path to save the CSV log of training history
csv_log_save_path = '/content/drive/MyDrive/Colab Notebooks/training_history_ResNet50_3.csv'

# Initialize the ModelCheckpoint and CSVLogger callbacks
model_checkpoint = ModelCheckpoint(best_model_save_path,
                                   monitor='val_accuracy',
                                   mode='max',
                                   save_best_only=True,
                                   verbose=1)

csv_logger = CSVLogger(csv_log_save_path, separator=',', append=False)
# Load the best saved model
best_model_path = '/content/drive/MyDrive/Colab Notebooks/best_model_ResNet50.h5'

last_model_path = '/content/drive/MyDrive/Colab Notebooks/last_model_ResNet50.h5'


# Load the best saved model
model = load_model(last_model_path)

# Compile the model with a new learning rate
new_learning_rate = 0.01  # Replace with your desired rate
model.compile(optimizer=Adam(learning_rate=new_learning_rate),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])


# Continue training
history = model.fit(datagen.flow(X_train, y_train, batch_size=32),
                    epochs=400,  # specify the number of additional epochs
                    validation_data=(X_val, y_val),
                    callbacks=[model_checkpoint, csv_logger])

model.save('/content/drive/MyDrive/Colab Notebooks/last_model_ResNet50.h5')

Epoch 1/400
29/29 [==============================] - ETA: 0s - loss: 0.8701 - accuracy: 0.6515
Epoch 1: val_accuracy improved from -inf to 0.49837, saving model to /content/drive/MyDrive/Colab Notebooks/best_model_ResNet50.h5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


29/29 [==============================] - 20s 527ms/step - loss: 0.8701 - accuracy: 0.6515 - val_loss: 1.2226 - val_accuracy: 0.4984
Epoch 2/400
29/29 [==============================] - ETA: 0s - loss: 0.8583 - accuracy: 0.6428
Epoch 2: val_accuracy improved from 0.49837 to 0.70033, saving model to /content/drive/MyDrive/Colab Notebooks/best_model_ResNet50.h5
29/29 [==============================] - 13s 461ms/step - loss: 0.8583 - accuracy: 0.6428 - val_loss: 1.0070 - val_accuracy: 0.7003
Epoch 3/400
29/29 [==============================] - ETA: 0s - loss: 0.8627 - accuracy: 0.6515
Epoch 3: val_accuracy improved from 0.70033 to 0.71010, saving model to /content/drive/MyDrive/Colab Notebooks/best_model_ResNet50.h5
29/29 [==============================] - 14s 487ms/step - loss: 0.8627 - accuracy: 0.6515 - val_loss: 0.7467 - val_accuracy: 0.7101
Epoch 4/400
29/29 [==============================] - ETA: 0s - loss: 0.8562 - accuracy: 0.6526
Epoch 4: val_accuracy did not improve from 0.71010


KeyboardInterrupt: ignored